In [20]:
import pandas as pd
import mc_processor
import os

In [21]:
## toggle

# survey_year = input("What year is the survey you want to analyze from? (2022, 2023, or 2024)")
survey_year = str(2024)

In [22]:
# read results files

# clarifier = "q4repeated/"
clarifier = ""

codebook = pd.read_csv(os.path.join(survey_year,f'raw/{clarifier}levels_codebook.csv'), encoding='utf-8')
data = pd.read_csv(os.path.join(survey_year,f'raw/{clarifier}data.csv'), encoding='utf-8')
q_codebook = pd.read_csv(os.path.join(survey_year,f'raw/{clarifier}question_codebook.csv'), encoding='utf-8', index_col = 'qid')

# q_codebook into dict
q_codebook = q_codebook.to_dict().get('qidFull')

In [23]:
nulldata = pd.DataFrame(data.isnull().sum(),columns=["Sum_Null"]) ##2204 rows in 2024
nulldata.sort_values(by=["Sum_Null"],ascending=False,inplace=True)

# nulldata[nulldata["Sum_Null"]>0].head(20)

Use `get_percents` to get percent who selected each option, works for questions that have multiple or single selection.

Demo input optional. For all demo categories, see next cell.

In [24]:
# all demo categories
for key in q_codebook:
    if "dem" in key:
        print(key)

xdemAll
demZIP
xdemGender
demAgeGeneration
xdemIdeo3
xdemInc3
xdemWhite
xdemHispBin
demBlackBin
demRaceOther
xdemReligion
xdemReligOther
xdemEvang
xdemUsr
xdemEmploy
xdemMilHH1
xdemBidenApprove
xdemBidenApprove2
demInsured
demInsType
demUnion
demLGBTQ1
xdemMarried
BPCxdem1
BPCxdem2
BPCxdem3
BPCxdem4
BPCxdem5
demGender3
demRVoter


In [25]:
## FILTER DATA BY REGISTERED VOTER

data = data[data["demRVoter"]==1]

In [26]:
## OVERALL CONFIDENCE RATES

demo = None
question_number = 20

dfs = []
cols = []
for i in range(20,24):
    dfs.append(mc_processor.run_confidence(data,codebook,q_codebook,i,suppress_output=True).map(lambda x: f"{x:.1%}"))
    cols.append(q_codebook[f"BPC{i}a"])

df = pd.concat(dfs, axis=1)
df.columns = cols
df

,How confident are you that your vote in the November 2024 General Election will be counted as you intend?,How confident are you that votes in your county or city will be counted as intendedin the November 2024 General Election?,How confident are you that votes in your state will be counted as intended in the November 2024 General Election?,How confident are you that votes nationwide will be counted as intended in the November 2024 General Election?
Very confident,47.0%,45.3%,45.0%,38.7%
Somewhat confident,29.7%,32.0%,30.6%,31.3%
Not to confident,9.9%,9.9%,10.9%,13.9%
Don't know/ No opinion,7.5%,7.0%,7.2%,7.9%
Not confident at all,5.9%,5.8%,6.3%,8.2%


In [27]:
## TO ASK FOR CONFIDENCE QUESTIONS

#split sample a - regular order
#split sample b - reverse order
#split sample c - random order

all_dfs = []

for i in range(20,24):
    # store dfs for each sample
    q_dfs = []    
    for split_sample in ["a","b","c"]:
        question = "BPC"+str(i)+split_sample
    
        # get results for question by sample
        df = mc_processor.run_and_display(data,codebook,q_codebook,question,survey_year,demo=None,suppress_output=True)
        df.columns = [f"split sample {split_sample}"]
        q_dfs.append(df)
    
    print(q_codebook[question]) # defaults to split sample c, but tested with other a/b/c combos and got same result (they rotated question order but didn't change question #)
    
    q_df = pd.concat(q_dfs, axis=1)

    q_df["deviation"] = q_df.std(axis=1)

    # format first three q_df columns as percent
    q_df.iloc[:,0:3] = q_df.iloc[:,0:3].map(lambda x: f"{x:.1%}")

    display(q_df)

    all_dfs.append(q_df)


How confident are you that your vote in the November 2024 General Election will be counted as you intend?


,split sample a,split sample b,split sample c,deviation
Very confident,46.5%,42.3%,52.4%,0.050498
Somewhat confident,31.3%,31.7%,26.0%,0.032094
Don't know/ No opinion,7.9%,8.7%,6.0%,0.013987
Not to confident,7.9%,10.4%,11.2%,0.017544
Not confident at all,6.4%,6.8%,4.5%,0.012573


How confident are you that votes in your county or city will be counted as intendedin the November 2024 General Election?


,split sample a,split sample b,split sample c,deviation
Very confident,45.1%,41.7%,49.3%,0.038304
Somewhat confident,33.3%,31.2%,31.6%,0.010879
Not to confident,9.6%,10.7%,9.4%,0.007327
Don't know/ No opinion,7.1%,8.5%,5.4%,0.015527
Not confident at all,5.0%,7.9%,4.3%,0.018691


How confident are you that votes in your state will be counted as intended in the November 2024 General Election?


,split sample a,split sample b,split sample c,deviation
Very confident,45.4%,38.7%,51.3%,0.063023
Somewhat confident,31.3%,31.7%,28.7%,0.016299
Not to confident,10.8%,11.6%,10.2%,0.007390
Don't know/ No opinion,6.5%,9.4%,5.5%,0.020140
Not confident at all,6.0%,8.6%,4.3%,0.021355


How confident are you that votes nationwide will be counted as intended in the November 2024 General Election?


,split sample a,split sample b,split sample c,deviation
Very confident,35.7%,37.2%,43.3%,0.040411
Somewhat confident,33.5%,30.1%,30.2%,0.019277
Not to confident,14.2%,13.8%,13.7%,0.002852
Not confident at all,8.9%,9.2%,6.4%,0.015672
Don't know/ No opinion,7.6%,9.7%,6.3%,0.016929


In [28]:
# TO ASK ALL QUESTIONS FOR 2024 (sub confidence)

demo = None
questions = []

for i in range(1,20):
    questions.append("BPC"+str(i))
questions.append("BPC24") #add last q following confidence

for question in questions:
     df = mc_processor.run_and_display(data,codebook,q_codebook,question,survey_year,demo,suppress_output=False)

     # df bar chart
    #  df.T.plot(kind='bar', xlabel="Response", ylabel="Count", figsize=(10,5))



BPC1: How often do you seek out the following kinds of information about elections, if at all? 


,"Only around major election dates or deadlines (e.g. Voter registration deadlines, Election Day)",A few times in the weeks around major election dates or deadlines,Regularly in the weeks around major election dates or deadlines,Consistently throughout the year,Never
Information about how to register and vote,25%,14%,17%,16%,28%
Information about how elections are run,22%,18%,18%,18%,24%
Information about who won an election,33%,16%,21%,20%,9%
Information about candidates,17%,18%,27%,28%,10%
News about campaigns,16%,20%,24%,25%,15%




BPC2: Below is a list of sources from which people might get news and information about elections. After reading this list of sources, please choose the three that you use most often to learn about elections. 


,overall
National television,45%
Social media,38%
Local or regional television,35%
Search through Google or other search engines,30%
"National news outlets, print or online",30%
"Local or state news outlets, print or online",22%
Radio,14%
Podcasts,11%
"News aggregator apps (e.g. NewsBreak, Smart News, Apple News)",9%
"Newsletters, blogs, or online forums",6%




BPC3: In the United States, there is information voters need to register and vote. Where are you most likely to look for this information? Please select up to three. 


...

,overall
Social media platforms and government should share the responsibility,35%
Mainly the responsibility of social media platforms,26%
Mainly the responsibility of government,16%
Neither are responsible,11%
Don't know/ No opinion,11%




BPC19: Do you agree or disagree with the following statement?In the most recent election I voted in, my voting experience was mostly positive.


,overall
Strongly agree,47%
Somewhat agree,34%
Don't know/ No opinion,11%
Somewhat disagree,6%
Strongly disagree,2%




BPC24: How concerned are you, if at all, about the following when it comes to the 2024 General Election? 


,Very concerned,Somewhat concerned,Not too concerned,Not concerned at all,Don't know/ No opinion
Inaccurate or misleading information about elections,35%,35%,15%,8%,7%
Use of AI to spread disinformation or manipulate public opinion,37%,32%,14%,8%,9%
Interference from foreign entities (including countries),27%,34%,20%,11%,9%
"Counting ineligible votes (e.g. duplicate, non-citizen, or deceased voters)",29%,26%,19%,17%,8%
Attempts to overturn the results of a fair election,35%,29%,16%,11%,8%
Biased or inaccurate counting of ballots,28%,29%,20%,15%,8%
Illegal or improper use of mail-in ballots or drop boxes,29%,27%,21%,16%,7%
"Presence of guns, violence, or intimidation at voting locations",25%,30%,21%,17%,7%
Violence or civil unrest after election day,33%,32%,17%,10%,8%
Restrictive voting equipment,21%,28%,25%,15%,11%


MISC - Figuring out conjoint issue

In [29]:
# get columsn in data that start iwth "BPC6"
data[data.filter(regex='^BPC6').columns]

,BPC6_1NET,BPC6_2NET,BPC6_3NET,BPC6_4NET,BPC6_5NET,BPC6_6NET,BPC6_7NET,BPC6_8NET,BPC6_9NET,BPC6_10NET,...,BPC6_12NET,BPC6_13NET,BPC6_14NET,BPC6_15NET,BPC6_16NET,BPC6_17NET,BPC6_18NET,BPC6_19NET,BPC6_20NET,BPC6_18_TEXT
0,2,2,2,2,2,2,2,2,2,2,...,2,1,1,2,2,2,2,2,2,NaN
1,1,2,2,2,2,2,1,2,2,2,...,2,2,2,2,2,2,2,2,2,NaN
3,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,1,NaN
4,2,2,2,2,2,2,2,2,2,2,...,2,1,2,2,2,2,2,2,2,NaN
5,2,2,2,1,2,2,2,2,2,2,...,2,2,2,2,1,2,2,2,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,2,1,2,2,2,2,2,2,2,2,...,2,1,2,2,2,2,2,2,2,NaN
2212,2,2,2,2,2,2,2,2,2,2,...,2,1,2,2,2,2,2,2,2,NaN
2213,2,1,1,2,2,2,2,2,2,2,...,2,1,2,2,2,2,2,2,2,NaN
2215,1,2,2,2,2,2,2,2,2,2,...,2,1,1,2,2,2,2,2,2,NaN


In [31]:
q_columns = data.filter(regex='^'+question+"_").columns

data_type_check(data,q_columns)

data.filter(regex='^' + question + r'[a-z]$').columns

Index([], dtype='object')